In [ ]:
import os
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# -------------------------------------------------------
# 1. Basic paths (if needed)
# -------------------------------------------------------
PROJECT_ROOT = os.getcwd()
DB_PATH = os.path.join(PROJECT_ROOT, "Output", "database", "unified_database.db")

# df_seg = waypoint map-matched by segment (already computed)
# df_tp  = trajs cleaned from the DB (already loaded)

assert "df_seg" in globals(), "df_seg must exist (waypoint map-matched by segment)"
assert "df_tp"  in globals(), "df_tp must exist (trajs cleaned from the DB)"

# -------------------------------------------------------
# 2. Load SegmentId_to_link table from the DB
# -------------------------------------------------------
conn = sqlite3.connect(DB_PATH)
df_map = pd.read_sql("SELECT * FROM SegmentId_to_link", conn)
conn.close()

print("SegmentId_to_link columns:", df_map.columns.tolist())

# Robust detection of segment and link columns
seg_col_candidates  = [c for c in df_map.columns if "segment" in c.lower()]
link_col_candidates = [c for c in df_map.columns if "link" in c.lower()]

SEG_COL  = seg_col_candidates[0]
LINK_COL = link_col_candidates[0]

print("Using mapping columns -> segment:", SEG_COL, ", link:", LINK_COL)

# -------------------------------------------------------
# 3. Prepare traj speeds by link (using the mapping)
# -------------------------------------------------------
# Keep only necessary columns
df_tp_link = df_tp[["SegmentId", "CrossingSpeedMph"]].copy()

# Clean speeds
df_tp_link["CrossingSpeedMph"] = pd.to_numeric(df_tp_link["CrossingSpeedMph"], errors="coerce")
df_tp_link = df_tp_link.replace([np.inf, -np.inf], np.nan).dropna(subset=["CrossingSpeedMph"])
df_tp_link = df_tp_link[(df_tp_link["CrossingSpeedMph"] > 0) & (df_tp_link["CrossingSpeedMph"] < 120)]

# Join with mapping table SegmentId -> link_id
df_map_use = df_map[[SEG_COL, LINK_COL]].copy()
df_map_use[SEG_COL] = df_map_use[SEG_COL].astype(df_tp_link["SegmentId"].dtype)

df_tp_link = df_tp_link.merge(
    df_map_use,
    left_on="SegmentId",
    right_on=SEG_COL,
    how="inner"
)

# Average traj speed per link_id
tp_link_speed = (
    df_tp_link
    .groupby(LINK_COL)["CrossingSpeedMph"]
    .mean()
    .reset_index()
    .rename(columns={LINK_COL: "link_id", "CrossingSpeedMph": "speed_tp"})
)

print("Trajs per-link rows:", len(tp_link_speed))

# -------------------------------------------------------
# 4. Prepare waypoint speeds by link (df_seg already has link_id)
# -------------------------------------------------------
df_wp_link = df_seg[["link_id", "speed_mph"]].copy()
df_wp_link["speed_mph"] = pd.to_numeric(df_wp_link["speed_mph"], errors="coerce")
df_wp_link = df_wp_link.replace([np.inf, -np.inf], np.nan).dropna(subset=["speed_mph"])
df_wp_link = df_wp_link[(df_wp_link["speed_mph"] > 0) & (df_wp_link["speed_mph"] < 120)]

wp_link_speed = (
    df_wp_link
    .groupby("link_id")["speed_mph"]
    .mean()
    .reset_index()
    .rename(columns={"speed_mph": "speed_wp"})
)

print("Waypoint per-link rows:", len(wp_link_speed))

# -------------------------------------------------------
# 5. Merge by link_id -> final dataset for validation
# -------------------------------------------------------
# Unify link_id types
wp_link_speed["link_id"] = wp_link_speed["link_id"].astype(str)
tp_link_speed["link_id"] = tp_link_speed["link_id"].astype(str)

df_val = wp_link_speed.merge(tp_link_speed, on="link_id", how="inner")

print("Validation rows (links with both sources):", len(df_val))
print(df_val.head())

x = df_val["speed_wp"].values   # waypoint map-matched avg speed per link
y = df_val["speed_tp"].values   # trajs cleaned avg speed per link

# -------------------------------------------------------
# 6. Metrics: RMSE, MAPE, R²
# -------------------------------------------------------
rmse = np.sqrt(np.mean((y - x) ** 2))

# R²
corr = np.corrcoef(x, y)[0, 1]
r2 = corr ** 2

# MAPE (note: can be large if there are very low speeds)
mape = np.mean(np.abs((y - x) / x)) * 100

print(f"RMSE = {rmse:.2f} mph, MAPE = {mape:.2f} %, R² = {r2:.3f}")

# -------------------------------------------------------
# 7. Scatterplot (cross-source validation)
# -------------------------------------------------------
plt.figure(figsize=(10, 7))

plt.scatter(
    x, y,
    s=50,
    color="lightcoral",
    edgecolor="black",
    alpha=0.7
)

# 1:1 line
min_val = min(x.min(), y.min())
max_val = max(x.max(), y.max())
plt.plot([min_val, max_val], [min_val, max_val], "r--", label="1:1 Line")

plt.xlabel("Waypoint Map-Matched Speed (mph)", fontsize=14)
plt.ylabel("Trajs (cleaned + mapped) Speed (mph)", fontsize=14)
plt.title("Cross-Source Speed Validation: Waypoint Map-Matched vs Trajs (cleaned + mapped)",
          fontsize=16)

metrics_text = f"RMSE: {rmse:.2f} mph\nMAPE: {mape:.2f}%\nR²: {r2:.3f}"
plt.text(
    0.05, 0.95, metrics_text,
    transform=plt.gca().transAxes,
    fontsize=12,
    verticalalignment="top",
    bbox=dict(facecolor="lightyellow", alpha=0.8, edgecolor="gray")
)

plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()

out_dir = os.path.join(PROJECT_ROOT, "figures_from_tool_db_only")
os.makedirs(out_dir, exist_ok=True)
out_path = os.path.join(out_dir, "cross_source_speed_validation_links.png")
plt.savefig(out_path, dpi=300)
plt.show()

print("Figure saved to:", out_path)
